In [56]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import (StructType, StructField, StringType,
                               DoubleType, IntegerType, LongType,DateType)
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark import SparkConf
from pyspark import SparkContext
import multiprocessing



In [2]:
spark = SparkSession.builder.appName("Microsoft_Kaggle").getOrCreate()


In [5]:
data = spark.read.csv("../data/train.csv",inferSchema=True,header=True)

In [6]:
data.show()

+--------------------+------------+-------------+---------------+------------+------+----------------+----------------+-------------------------+-------------------------+-------------------+-----------------+------+-----------------+--------------+----------------------+-----------------+---------------------------+---------+---------+--------+-------+-------+--------------------+--------------------+----------+-----------+---------------+-------+-----+---------------+------------+--------+------------+---------------------+-------------------+------------------------+-------------------------+-------------------------+--------------------------------------+-------------------------------+---------------------+-------------------------------+--------------------------+--------------------------------+--------------------------+-----------------------+----------------------+-------------------------------------------------+-------------------------------------------------+-------------

In [12]:
cols_imp = ['MachineIdentifier','AvSigVersion','Platform','Census_OSBranch']

In [13]:
data.filter(col('AvSigVersion')=='1.155.266.0').select(cols_imp).show(truncate=False)

+--------------------------------+------------+--------+---------------+
|MachineIdentifier               |AvSigVersion|Platform|Census_OSBranch|
+--------------------------------+------------+--------+---------------+
|244fa807a7c759a3a55f10d61d7a0f0c|1.155.266.0 |windows8|rs4_release    |
|78523fe75bdf1c49eba244a8da134bcc|1.155.266.0 |windows8|th1            |
|7f356b3f062302c10c53948a9e1c0c20|1.155.266.0 |windows8|rs4_release    |
|82f123c563221246782eb2a85e3daf21|1.155.266.0 |windows8|th2_release    |
|b93bcf439cd83e1426bb2094de70a796|1.155.266.0 |windows8|th2_release    |
|bbc2d7f48f578935fd7033731d1bde5c|1.155.266.0 |windows8|rs1_release    |
|cb58923b451b201e9de5668f186879a5|1.155.266.0 |windows8|th2_release_sec|
|e5037fe1a117c4edc98ca4bb600491e1|1.155.266.0 |windows8|rs4_release    |
|e7d094c26d231954d945865f198e5233|1.155.266.0 |windows8|rs1_release    |
|e9ac07c5b6a787580048aea53f73f363|1.155.266.0 |windows8|th2_release    |
+--------------------------------+------------+----

In [17]:
data_horas = spark.read.csv('fechas.csv',inferSchema=True,header=True)

In [18]:
data_horas.show()

+-------------------+---------------+
|              fecha|release_version|
+-------------------+---------------+
|2013-07-18 09:08:00|    1.155.266.0|
|2014-02-21 14:20:00|    1.167.387.0|
|2014-04-03 01:17:00|   1.169.1625.0|
|2014-04-12 17:16:00|   1.169.2478.0|
|2014-03-18 01:09:00|     1.169.55.0|
|2014-06-19 17:16:00|    1.177.229.0|
|2014-11-01 03:19:00|   1.187.1121.0|
|2014-10-23 01:07:00|    1.187.322.0|
|2014-11-19 00:14:00|    1.189.189.0|
|2014-12-25 12:16:00|    1.191.897.0|
|2015-02-18 09:32:00|    1.193.185.0|
|2015-03-06 12:55:00|   1.193.1928.0|
|2015-03-21 07:28:00|   1.193.3377.0|
|2015-03-21 12:34:00|   1.193.3388.0|
|2015-02-20 00:51:00|    1.193.418.0|
|2015-03-24 12:22:00|    1.195.109.0|
|2015-03-31 17:38:00|   1.195.1107.0|
|2015-04-07 12:11:00|   1.195.2226.0|
|2015-04-13 00:14:00|   1.195.3092.0|
|2015-04-14 00:18:00|   1.195.3225.0|
+-------------------+---------------+
only showing top 20 rows



In [80]:
data_com = data.join(data_horas,data['AvSigVersion'] == data_horas["release_version"],'left')\
            .withColumnRenamed('fecha','AvSigVersionDate')\
            .withColumnRenamed('release_version','Borrar')

In [28]:
cols_imp=cols_imp+['AvSigVersionDate']

In [29]:
data_com.select(cols_imp).show(10,False)

+--------------------------------+------------+---------+----------------------+-------------------+
|MachineIdentifier               |AvSigVersion|Platform |Census_OSBranch       |AvSigVersionDate   |
+--------------------------------+------------+---------+----------------------+-------------------+
|0000028988387b115f69f31a3bf04f09|1.273.1735.0|windows10|rs4_release           |2018-08-20 21:11:00|
|000007535c3f730efa9ea0b7ef1bd645|1.263.48.0  |windows10|rs4_release           |2018-03-01 22:10:00|
|000007905a28d863f6d0d597892cd692|1.273.1341.0|windows10|rs4_release           |2018-08-13 23:35:00|
|00000b11598a75ea8ba1beea8459149f|1.273.1527.0|windows10|rs4_release           |2018-08-17 03:01:00|
|000014a5f00daa18e76b81417eeb99fc|1.273.1379.0|windows10|rs4_release           |2018-08-14 12:33:00|
|000016191b897145d069102325cab760|1.273.1094.0|windows10|rs4_release           |2018-08-09 06:59:00|
|0000161e8abf8d8b89c5ab8787fd712b|1.273.845.0 |windows10|rs4_release           |2018-08-05 

In [47]:
cols_imp = ['MachineIdentifier',
 'AvSigVersion',
 'AvSigVersionDate',
 'AvSigVersion_0',
 'AvSigVersion_1',
 'AvSigVersion_2',
 'AvSigVersion_3']

In [81]:
data_com = data_com.withColumn('AvSigVersion_0', split(data_com['AvSigVersion'], '\.')[0].cast(IntegerType()))\
                    .withColumn('AvSigVersion_1', split(data_com['AvSigVersion'], '\.')[1].cast(IntegerType()))\
                    .withColumn('AvSigVersion_2', split(data_com['AvSigVersion'], '\.')[2].cast(IntegerType()))\
                    .withColumn('AvSigVersion_3', split(data_com['AvSigVersion'], '\.')[3].cast(IntegerType()))
data_com.select(cols_imp).show(10)

+--------------------+------------+-------------------+--------------+--------------+--------------+--------------+
|   MachineIdentifier|AvSigVersion|   AvSigVersionDate|AvSigVersion_0|AvSigVersion_1|AvSigVersion_2|AvSigVersion_3|
+--------------------+------------+-------------------+--------------+--------------+--------------+--------------+
|0000028988387b115...|1.273.1735.0|2018-08-20 21:11:00|             1|           273|          1735|             0|
|000007535c3f730ef...|  1.263.48.0|2018-03-01 22:10:00|             1|           263|            48|             0|
|000007905a28d863f...|1.273.1341.0|2018-08-13 23:35:00|             1|           273|          1341|             0|
|00000b11598a75ea8...|1.273.1527.0|2018-08-17 03:01:00|             1|           273|          1527|             0|
|000014a5f00daa18e...|1.273.1379.0|2018-08-14 12:33:00|             1|           273|          1379|             0|
|000016191b897145d...|1.273.1094.0|2018-08-09 06:59:00|             1|  

In [50]:
data_com.select('AvSigVersion_0').distinct().show()

+--------------+
|AvSigVersion_0|
+--------------+
|             1|
|             0|
+--------------+



In [54]:
data_com.select('AvSigVersion_1').distinct().count()

51

In [55]:
data_com.select('AvSigVersion_2').distinct().count()

2766

In [51]:
data_com.select('AvSigVersion_3').distinct().show()

+--------------+
|AvSigVersion_3|
+--------------+
|             0|
+--------------+



In [70]:
data_com.select('AvSigVersionDate').show()

+-------------------+
|   AvSigVersionDate|
+-------------------+
|2018-08-20 21:11:00|
|2018-03-01 22:10:00|
|2018-08-13 23:35:00|
|2018-08-17 03:01:00|
|2018-08-14 12:33:00|
|2018-08-09 06:59:00|
|2018-08-05 00:52:00|
|2018-08-14 20:09:00|
|2018-09-10 00:51:00|
|2018-08-07 09:14:00|
|2018-08-05 12:25:00|
|2018-08-22 05:11:00|
|2018-08-14 10:21:00|
|2018-09-07 10:38:00|
|2018-08-06 15:12:00|
|2018-08-24 23:45:00|
|2018-08-28 06:37:00|
|2018-07-28 04:04:00|
|2018-09-25 16:42:00|
|2018-08-21 17:44:00|
+-------------------+
only showing top 20 rows



In [83]:
w = Window.partitionBy().orderBy('AvSigVersion_0','AvSigVersion_1','AvSigVersion_2','AvSigVersion_3')

In [87]:
data_com = data_com.withColumn('AvSigVersionDate',col('AvSigVersionDate').cast(DateType()))

In [88]:
data_windows = data_com.withColumn('date_lag',lead('AvSigVersionDate').over(w))

In [85]:
data_windows.persist()
data_windows.count()

8921483

In [66]:
cols_imp

['MachineIdentifier',
 'AvSigVersion',
 'AvSigVersionDate',
 'AvSigVersion_0',
 'AvSigVersion_1',
 'AvSigVersion_2',
 'AvSigVersion_3']

In [91]:
data_windows
.withColumn('date_lead_num', datediff(col('date_lag'),col('AvSigVersionDate')))
.select(cols_imp+['date_lead_num'])
.filter(col('AvSigVersion_0')>0)
.show()

+--------------------+-----------------+----------------+--------------+--------------+--------------+--------------+-------------+
|   MachineIdentifier|     AvSigVersion|AvSigVersionDate|AvSigVersion_0|AvSigVersion_1|AvSigVersion_2|AvSigVersion_3|date_lead_num|
+--------------------+-----------------+----------------+--------------+--------------+--------------+--------------+-------------+
|9686a9afcb31e2160...|1.2&#x17;3.1144.0|      2018-08-10|             1|          null|          1144|             0|        -1849|
|78523fe75bdf1c49e...|      1.155.266.0|      2013-07-18|             1|           155|           266|             0|            0|
|b93bcf439cd83e142...|      1.155.266.0|      2013-07-18|             1|           155|           266|             0|            0|
|7f356b3f062302c10...|      1.155.266.0|      2013-07-18|             1|           155|           266|             0|            0|
|bbc2d7f48f578935f...|      1.155.266.0|      2013-07-18|             1|    